In [1]:
import sys
sys.path.append("../../")
from Package.DQuadratization import *
from Package.EquationSystem import *
from Package.Combinations import *
from qbee import *
import sympy as sp
from Package.Simulation.numerical import *
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

/Users/yubocai/anaconda3/envs/mlp/lib/python3.9/site-packages/qbee/util.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Multi-stability in the system

---

Consider the following system of ODEs:
$$
X’ = -x (x - a_1)(x - a_2) \ldots (x - a_n)
$$
where $a_{1}, \ldots, a_{n}$ are positive numbers. This system has $n+1$ equilibria: $x = 0, a_{1}, \ldots, a_{n}$. Let's take $n = 10$ and $a_{1} = 1, \ldots, a_{10} = 10$.

In [13]:
x = sp.Symbol('x')
num_list = list(range(1, 8))

rhs = - x 
for i in num_list:
    rhs = rhs * (x - i)

system = [sp.Eq(x, rhs)]
multistable_eq_system = EquationSystem(system)

In [14]:
display(multistable_eq_system.show_system_latex())

<IPython.core.display.Latex object>

In [15]:
inner_quadratization = optimal_inner_quadratization(multistable_eq_system)

The Original System is: 


<IPython.core.display.Latex object>

The Optimal Dissipative Quadratization is: 


<IPython.core.display.Latex object>

The new introduced variables are: 


<IPython.core.display.Latex object>

The Optimal Quadratic Dissipative System is (with substitution): 


<IPython.core.display.Latex object>

In [16]:
print(inner_quadratization[3])

{w1: x**3, w2: x**6, w3: x**2, w4: x**4, w5: x**7}


### check the equilibria is dissipative or not

---

In [17]:
rhs_differential = sp.diff(rhs, x, 1).expand()
display(rhs_differential)
dissipative_equilibrium_list = []
for i in range(1, 8):
    if rhs_differential.subs(x, i) < 0:
        dissipative_equilibrium_list.append([i])
print("Dissipative equilibrium points: ", dissipative_equilibrium_list)

-8*x**7 + 196*x**6 - 1932*x**5 + 9800*x**4 - 27076*x**3 + 39396*x**2 - 26136*x + 5040

Dissipative equilibrium points:  [[1], [3], [5], [7]]


In [18]:
dissipative_multi_system = dquadratization_multi_equilibrium(multistable_eq_system, dissipative_equilibrium_list, method='numpy')

-------------------------- Compute a quadratization dissipative with multi given equilibrium --------------------------
The original system is: 


<IPython.core.display.Latex object>

-------------------------- The Dissipative quadratized system --------------------------


<IPython.core.display.Latex object>

-------------------------- The lambda value --------------------------
The lambda value is:  67108864
-------------------------- The Dissipative quadratized system with lambda value --------------------------


<IPython.core.display.Latex object>

-------------------------- The Jacobian matrix --------------------------


<IPython.core.display.Latex object>

In [19]:
dissipative_multi_system = dquadratization_multi_equilibrium(multistable_eq_system, dissipative_equilibrium_list, method='Routh-Hurwitz')

-------------------------- Compute a quadratization dissipative with multi given equilibrium --------------------------
The original system is: 


<IPython.core.display.Latex object>

-------------------------- The Dissipative quadratized system --------------------------


<IPython.core.display.Latex object>

-------------------------- The lambda value --------------------------
The lambda value is:  134217728
-------------------------- The Dissipative quadratized system with lambda value --------------------------


<IPython.core.display.Latex object>

-------------------------- The Jacobian matrix --------------------------


<IPython.core.display.Latex object>

In [20]:
dissipative_multi_system = dquadratization_multi_equilibrium(multistable_eq_system, dissipative_equilibrium_list, method='sympy-naive')

-------------------------- Compute a quadratization dissipative with multi given equilibrium --------------------------
The original system is: 


<IPython.core.display.Latex object>

-------------------------- The Dissipative quadratized system --------------------------


<IPython.core.display.Latex object>

-------------------------- The lambda value --------------------------
The lambda value is:  67108864
-------------------------- The Dissipative quadratized system with lambda value --------------------------


<IPython.core.display.Latex object>

-------------------------- The Jacobian matrix --------------------------


<IPython.core.display.Latex object>

### Smaller degree testing example

---
Consider the following system of ODEs:
$$
x' = -x(x - a)(x + a)
$$

In [7]:
x = sp.Symbol('x')

system_eq_a4 = EquationSystem([sp.Eq(x, -x * (x - 4) * (x + 4))])
system_eq_a10 = EquationSystem([sp.Eq(x, -x * (x - 10) * (x + 10))])
system_eq_a20 = EquationSystem([sp.Eq(x, -x * (x - 20) * (x + 20))])
system_eq_a10000 = EquationSystem([sp.Eq(x, -x * (x - 10000) * (x + 10000))])

In [4]:
dissipative_system_eq_a4 = dquadratization_multi_equilibrium(system_eq_a4, [[4],[-4]], method='numpy', display=False)
dissipative_system_eq_a4[0]

1

In [12]:
inner_system_eq_a10000 = optimal_inner_quadratization(system_eq_a10000)

The Original System is: 


<IPython.core.display.Latex object>

The Optimal Dissipative Quadratization is: 


<IPython.core.display.Latex object>

The new introduced variables are: 


<IPython.core.display.Latex object>

The Optimal Quadratic Dissipative System is (with substitution): 


<IPython.core.display.Latex object>

In [10]:
dissipative_system_eq_a10000 = dquadratization_multi_equilibrium(system_eq_a10000, [[10000],[-10000]], method='numpy', display=False)
dissipative_system_eq_a10000[0]

1

In [13]:
dissipative_system_eq_a10000 = dquadratization_multi_equilibrium(system_eq_a10000, [[10000],[-10000]], method='sympy-naive', display=False)
dissipative_system_eq_a10000[0]

1

In [14]:
dissipative_system_eq_a10000 = dquadratization_multi_equilibrium(system_eq_a10000, [[10000],[-10000]], method='Routh-Hurwitz', display=False)
dissipative_system_eq_a10000[0]

1